In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np

In [26]:
#define tensors in pytorch - same as arrays in numpy - with tensor
X = torch.tensor(([2,9], [1,5], [3,6]), dtype=torch.float) #3x2 tensor
y = torch.tensor(([92], [100], [89]), dtype=torch.float) #3 by 1 tensor
x_test = torch.tensor(([4, 8]), dtype=torch.float)

#can check the size with .size()
X.size()
print(x_test)

tensor([4., 8.])


In [27]:
#apply scaling
#max function returns max element and indice
X_max, _ = torch.max(X, 0)
xPredicted_max, _ = torch.max(x_test, 0)

#divides all the elements by X_max - in this case X_max return largest single value in the matrices i.e. 9
X = torch.div(X, X_max)
x_test = torch.div(x_test, xPredicted_max)
y = y/100
print(X)
print(x_test)

tensor([[0.6667, 1.0000],
        [0.3333, 0.5556],
        [1.0000, 0.6667]])
tensor([0.5000, 1.0000])


In [28]:
#Building The Neural Network
#nn.module is the base class for any module in PyTorch
class NeuralNetwork(nn.Module):
    def __init__(self,):
        super(NeuralNetwork, self).__init__()
        self.inputSize = 2
        self.outputSize = 1
        self.hiddenNodes = 3
        
        self.W1 = torch.randn(self.inputSize, self.hiddenNodes) #Tensor is 2 by 3 since (1 by 2) * (2 by 3) gives desired (1 by 3)
        self.W2 = torch.randn(self.hiddenNodes, self.outputSize) #Tesnsor is 3 by 1
        
    def forward(self, X):
        self.z = torch.matmul(X, self.W1)
        self.z2 = self.sigmoid(self.z)
        self.z3 = torch.matmul(self.z2, self.W2)
        o = self.sigmoid(self.z3)
        return o
        
    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1-s)
    
    def backward(self, x, y, o):
        self.o_error = y - o
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        #torch.t is the transpose operation
        #recall in the second wave of operations we do sigmoid(z2 * W2)
        #thus this can be represented as two nodes in a computation graph, the sigmoid and the matrix multiplication
        #partial derivative at the matrix multiplication level
        self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))
        #partial derivative at the sigmoid level
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
        self.W1 += torch.matmul(torch.t(X), self.z2_delta)
        self.W2 += torch.matmul(torch.t(self.z2), self.o_delta)
    
         
    def train(self, X, y):
        # forward + backward pass for training
        o = self.forward(X)
        self.backward(X, y, o)
        
    def saveWeights(self, model):
        # we will use the PyTorch internal storage functions
        torch.save(model, "NN")
        # you can reload model with all the weights and so forth with:
        # torch.load("NN")
        
    def predict(self):
        print ("Predicted data based on trained weights: ")
        print ("Input (scaled): \n" + str(x_test))
        print ("Output: \n" + str(self.forward(x_test)))
    

In [29]:
#Training
NN = NeuralNetwork()
for i in range(100):
    #NN(X) automatically calls the forward so there is no need to explicity call forward
    print ("#" + str(i) + " Loss: " + str(torch.mean((y - NN(X))**2).detach().item()))  # mean sum squared loss
    NN.train(X, y)
NN.saveWeights(NN)
NN.predict()

#0 Loss: 0.08693492412567139
#1 Loss: 0.07286923378705978
#2 Loss: 0.06154783442616463
#3 Loss: 0.052441343665122986
#4 Loss: 0.045097678899765015
#5 Loss: 0.03914613649249077
#6 Loss: 0.034290965646505356
#7 Loss: 0.030300229787826538
#8 Loss: 0.026993749663233757
#9 Loss: 0.02423200011253357
#10 Loss: 0.021906867623329163
#11 Loss: 0.019934270530939102
#12 Loss: 0.018248530104756355
#13 Loss: 0.016798006370663643
#14 Loss: 0.015541750937700272
#15 Loss: 0.014447162859141827
#16 Loss: 0.013488010503351688
#17 Loss: 0.012643102556467056
#18 Loss: 0.011895153671503067
#19 Loss: 0.011229979805648327
#20 Loss: 0.010635868646204472
#21 Loss: 0.010103111155331135
#22 Loss: 0.00962356198579073
#23 Loss: 0.009190385229885578
#24 Loss: 0.008797802962362766
#25 Loss: 0.008440916426479816
#26 Loss: 0.00811552070081234
#27 Loss: 0.007818029262125492
#28 Loss: 0.007545334752649069
#29 Loss: 0.007294775918126106
#30 Loss: 0.0070640030317008495
#31 Loss: 0.006851005833595991
#32 Loss: 0.006653991062